In [1]:
from sklearn.preprocessing import LabelEncoder
from bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing import sequence
import sklearn
import numpy as np
import re

In [2]:
seq_length = 10
pad_length = 5
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt0]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

def readmaf(start, end, filename):
    count = 0
    oldlist =[]
    align = AlignIO.parse(filename, "maf")
    for multiple_alignment in align:
        count = count+1
        if count >start and count<=end:
            oldlist.append(multiple_alignment)
        elif count>end:
            break
    return oldlist

#oldlist stores MSAs
oldlist = readmaf(0, 1000000, "data/chr2.anc.maf")
print(oldlist[1][1])
print(dir(oldlist[1]))
print(oldlist[1][0].id)
#print(oldlist[2])

ID: panTro4.chr2A
Name: panTro4.chr2A
Number of features: 0
/start=4703
/size=1
/strand=1
/srcSize=113622374
Seq('a', SingleLetterAlphabet())
['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alphabet', '_annotations', '_append', '_get_per_column_annotations', '_per_col_annotations', '_records', '_set_per_column_annotations', '_str_line', 'add_sequence', 'annotations', 'append', 'column_annotations', 'extend', 'format', 'get_alignment_length', 'sort']
hg38.chr2


In [3]:
print(len(oldlist))

1000000


In [4]:
#alignList stores alignments between direct ancestor and descendent
def getAlign(inputList, keyword):
    alignList = []
    zeros = ''
    for i in range(len(inputList)):
        # index for the dog
        indexDes = -1
        indexAnc = -1

        for j in range(len(inputList[i])):
            if len(inputList[i][j])>=10 and 'canFam3' in inputList[i][j].id  :
                indexDes = j
            elif len(inputList[i][j])>=10 and inputList[i][j].id == '_CMAOL': #'_RMPC' :
                indexAnc = j
        if indexDes!=-1 and indexAnc!=-1:
            alignment = MultipleSeqAlignment([
                    SeqRecord(inputList[i][indexDes].seq+zeros, id=inputList[i][indexDes].id),
                    SeqRecord(inputList[i][indexAnc].seq+zeros, id=inputList[i][indexAnc].id),
                    #SeqRecord(zeros+inputList[i][indexDes].seq, id=inputList[i][indexDes].id)
                ]
            )
            alignList.append(alignment)
    return alignList

def zeros(length):
    return "0"*length
    
FinalList = []    
anc = []
des = []
keyword = ["_HP","_RM","_FC","_CS","_BO","_PB","_TO","_VC", "_PP"]
alignList = getAlign(oldlist, keyword)
print(len(alignList))
print(alignList[126])
print(len(alignList[126][1].seq))
print(len(alignList[126][1]))
for i in range (100):
    print('printing new alignment -------------------------------------------\n')
    print(alignList[i])

#print(str(alignList[62][1]))
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

357202
SingleLetterAlphabet() alignment with 2 rows and 20 columns
GATG---------------- canFam3.chr17
GATG---------------- _CMAOL
20
20
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 34 columns
AAGAGAG-AAACTAAAGCTGAGCCATAGGCTACA canFam3.chr17
AAGACAG-AAACTAAAGCTGAGCCATAGGCTACA _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 238 columns
A----------CTGACCTGAC----AGTCAGAGGAA-AGCAGGG...--- canFam3.chr17
A----------CTGATCTGACAATAAGTCAGGGGAA-AGAAGGG...--- _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 79 columns
------------------------------------------AG...TTT canFam3.chr17
------------------------------------------AG...TTT _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 14 columns
GCATTGTGTG---T

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','0', 'z']))
encoder = OneHotEncoder(sparse=False, dtype=int, categories=[range(6)])

def ungap(strX, strY):
    new_X = ''
    new_Y = ''
    for i in range(len(strX)):
        if strX[i] == '-' and strY[i] == '-':
            continue
        else :
            new_X = new_X+strX[i]
            new_Y = new_Y+strY[i]

    return new_X, new_Y
    

def getXY(alignList):
    listX = []
    listY = []
    for i in range(len(alignList)):
        X, Y = ungap(str(alignList[i][1].seq).upper(), str(alignList[i][0].seq).upper())
        listX.append(X)
        listY.append(Y)
    return listX, listY

def zeroPad(A):
    zeros = '0'*(pad_length+seq_length-len(A))
    A = A+zeros
    return A

def tokenize(listX, listY):
    X_token =[]
    X1_token = []
    Y_token =[]
    Y1_token = []
    for i in range(len(listX)):
        if len(listX[i])==seq_length:
            X = listX[i].replace('-','')
            Y = listY[i]
            X = zeroPad(X)
            Y = zeroPad(Y)
            X1 = '0'+X[:seq_length+pad_length-1]
            Y1 = '0'+Y[:seq_length+pad_length-1]
            X_token.append(X)
            Y_token.append(Y)
            X1_token.append(X1)
            Y1_token.append(Y1)
        elif len(listX[i])>seq_length:
            for j in range(len(listX[i])-seq_length):
                X = listX[i][j:j+seq_length].replace('-','')
                Y = listY[i][j:j+seq_length]
                X = zeroPad(X)
                Y = zeroPad(Y)
                X1 = '0'+X[:seq_length+pad_length-1]
                Y1 = '0'+Y[:seq_length+pad_length-1]
                X_token.append(X)
                Y_token.append(Y)
                X1_token.append(X1)
                Y1_token.append(Y1)
    return X_token, Y_token, X1_token, Y1_token

def one_hot_encoder(my_array, encoder):
    integer_encoded = label_encoder.transform(my_array)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = encoder.fit_transform(integer_encoded)
    #print(np.array(onehot_encoded))
    return onehot_encoded

def encode_list(X, Y, encoder):
    new_X =[]
    new_Y =[]
    for x, y in zip(X,Y):
        #new_X = np.append(new_X, one_hot_encoder(string_to_array(x), encoder))
        #new_Y = np.append(new_Y, one_hot_encoder(string_to_array(y), encoder))
        new_X.extend(one_hot_encoder(string_to_array(x), encoder))
        new_Y.extend(one_hot_encoder(string_to_array(y), encoder))
    
    return np.array(new_X).reshape((-1,seq_length+pad_length,6)), np.array(new_Y).reshape((-1,seq_length+pad_length,6))

X, Y=getXY(alignList)
X, Y, X1, Y1= tokenize(X,Y)
#print(X,Y)
X, Y= encode_list(X, Y, encoder)
X1, Y1= encode_list(X1, Y1, encoder)
for i in range(100):
    print(X[i], Y[i])

def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [1, 0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [0, 1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 0, 1]:
            result=result+"-"
    return result

def decoderY(array):
    result = ""    
    if array.tolist() == [1, 0, 0, 0, 0, 0]:
        result=result+"0" 
    elif array.tolist() == [0, 1, 0, 0, 0, 0]:
        result=result+"A"
    elif array.tolist() == [0, 0, 1, 0, 0, 0]:
        result=result+"C"
    elif array.tolist() == [0, 0, 0, 1, 0, 0]:
        result=result+"G"
    elif array.tolist() == [0, 0, 0, 0, 1, 0]:
        result=result+"T"
    elif array.tolist() == [0, 0, 0, 0, 0, 1]:
        result=result+"-"
    return result
    
#print(Y.shape)
#print(len(X))

[[0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]] [[0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]
[[0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]] [[0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]
[[0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 

In [11]:
pt1 = 9950000
pt2 = 10000000
pt3 = 10050000
X_train = X[:pt1]
X_val = X[pt1:pt2]
X_test = X[pt2:pt3]
y_train = Y[:pt1]
y_val =  Y[pt1:pt2]
y_test = Y[pt2:pt3]
# X_train1 = X1[:pt1]
# X_val1 = X1[pt1:pt2]
# X_test1 = X1[pt2:pt3]
y_train1 = Y1[:pt1]
y_val1 =  Y1[pt1:pt2]
y_test1 = Y1[pt2:pt3]

# X_train = X[:20000]
# X_val = X[20000:23000]
# X_test = X[23000:26000]
# y_train = Y[:20000]
# y_val =  Y[20000:23000]
# y_test = Y[23000:26000]

np.save('prepData/X_train_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), X_train)
np.save('prepData/X_val_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), X_val)
np.save('prepData/X_test_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), X_test)
np.save('prepData/y_train_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_train)
np.save('prepData/y_val_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_val)
np.save('prepData/y_test_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_test)

np.save('prepData/y_train1_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_train1)
np.save('prepData/y_val1_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_val1)
np.save('prepData/y_test1_gap_pad_camFam3_v3_chr2_size{}'.format(seq_length), y_test1)

In [7]:

X = np.array(X)
Y = np.array(Y)
#X1 = np.array(X1)
Y1 = np.array(Y1)
print(X)
print(X.shape)
print(X[0].shape)
print(X[0][0].shape)
print(type(X))
#print(X)

[[[0 1 0 0 0 0]
  [0 1 0 0 0 0]
  [0 0 0 1 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]

 [[0 1 0 0 0 0]
  [0 0 0 1 0 0]
  [0 1 0 0 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]

 [[0 0 0 1 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]

 ...

 [[0 0 0 0 1 0]
  [0 1 0 0 0 0]
  [0 1 0 0 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]

 [[0 1 0 0 0 0]
  [0 1 0 0 0 0]
  [0 1 0 0 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]

 [[0 1 0 0 0 0]
  [0 1 0 0 0 0]
  [0 1 0 0 0 0]
  ...
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]
  [1 0 0 0 0 0]]]
(10053269, 15, 6)
(15, 6)
(6,)
<class 'numpy.ndarray'>


In [8]:
# import difflib
# sm=difflib.SequenceMatcher(None,X_test,X_train)
# sm2=difflib.SequenceMatcher(None,y_test,y_train)
for i in range(100):
    print(i, decoder(X[i]))

0 AAGACAGAAA00000
1 AGACAGAAAC00000
2 GACAGAAACT00000
3 ACAGAAACTA00000
4 CAGAAACTAA00000
5 AGAAACTAAA00000
6 GAAACTAAAG00000
7 AAACTAAAGC00000
8 AACTAAAGCT00000
9 ACTAAAGCTG00000
10 CTAAAGCTGA00000
11 TAAAGCTGAG00000
12 AAAGCTGAGC00000
13 AAGCTGAGCC00000
14 AGCTGAGCCA00000
15 GCTGAGCCAT00000
16 CTGAGCCATA00000
17 TGAGCCATAG00000
18 GAGCCATAGG00000
19 AGCCATAGGC00000
20 GCCATAGGCT00000
21 CCATAGGCTA00000
22 CATAGGCTAC00000
23 ACTGATCTGA00000
24 CTGATCTGAC00000
25 TGATCTGACA00000
26 GATCTGACAA00000
27 ATCTGACAAT00000
28 TCTGACAATA00000
29 CTGACAATAA00000
30 TGACAATAAG00000
31 GACAATAAGT00000
32 ACAATAAGTC00000
33 CAATAAGTCA00000
34 AATAAGTCAG00000
35 ATAAGTCAGG00000
36 TAAGTCAGGG00000
37 AAGTCAGGGG00000
38 AGTCAGGGGA00000
39 GTCAGGGGAA00000
40 TCAGGGGAAA00000
41 CAGGGGAAAG00000
42 AGGGGAAAGA00000
43 GGGGAAAGAA00000
44 GGGAAAGAAG00000
45 GGAAAGAAGG00000
46 GAAAGAAGGG00000
47 AAAGAAGGGC00000
48 AAGAAGGGCA00000
49 AGAAGGGCAC00000
50 GAAGGGCACA00000
51 AAGGGCACAG00000
52 AGGGCACAGA00000
53 

In [9]:
for i in range(100):
    print(i, decoder(Y[i]))

0 AAGAGAGAAA00000
1 AGAGAGAAAC00000
2 GAGAGAAACT00000
3 AGAGAAACTA00000
4 GAGAAACTAA00000
5 AGAAACTAAA00000
6 GAAACTAAAG00000
7 AAACTAAAGC00000
8 AACTAAAGCT00000
9 ACTAAAGCTG00000
10 CTAAAGCTGA00000
11 TAAAGCTGAG00000
12 AAAGCTGAGC00000
13 AAGCTGAGCC00000
14 AGCTGAGCCA00000
15 GCTGAGCCAT00000
16 CTGAGCCATA00000
17 TGAGCCATAG00000
18 GAGCCATAGG00000
19 AGCCATAGGC00000
20 GCCATAGGCT00000
21 CCATAGGCTA00000
22 CATAGGCTAC00000
23 ACTGACCTGA00000
24 CTGACCTGAC00000
25 TGACCTGAC-00000
26 GACCTGAC--00000
27 ACCTGAC---00000
28 CCTGAC----00000
29 CTGAC----A00000
30 TGAC----AG00000
31 GAC----AGT00000
32 AC----AGTC00000
33 C----AGTCA00000
34 ----AGTCAG00000
35 ---AGTCAGA00000
36 --AGTCAGAG00000
37 -AGTCAGAGG00000
38 AGTCAGAGGA00000
39 GTCAGAGGAA00000
40 TCAGAGGAAA00000
41 CAGAGGAAAG00000
42 AGAGGAAAGC00000
43 GAGGAAAGCA00000
44 AGGAAAGCAG00000
45 GGAAAGCAGG00000
46 GAAAGCAGGG00000
47 AAAGCAGGGC00000
48 AAGCAGGGCA00000
49 AGCAGGGCAT00000
50 GCAGGGCATA00000
51 CAGGGCATAG00000
52 AGGGCATAGA00000
53 

In [10]:
for i in range(100):
    print(i, decoder(Y1[i]))

0 0AAGAGAGAAA0000
1 0AGAGAGAAAC0000
2 0GAGAGAAACT0000
3 0AGAGAAACTA0000
4 0GAGAAACTAA0000
5 0AGAAACTAAA0000
6 0GAAACTAAAG0000
7 0AAACTAAAGC0000
8 0AACTAAAGCT0000
9 0ACTAAAGCTG0000
10 0CTAAAGCTGA0000
11 0TAAAGCTGAG0000
12 0AAAGCTGAGC0000
13 0AAGCTGAGCC0000
14 0AGCTGAGCCA0000
15 0GCTGAGCCAT0000
16 0CTGAGCCATA0000
17 0TGAGCCATAG0000
18 0GAGCCATAGG0000
19 0AGCCATAGGC0000
20 0GCCATAGGCT0000
21 0CCATAGGCTA0000
22 0CATAGGCTAC0000
23 0ACTGACCTGA0000
24 0CTGACCTGAC0000
25 0TGACCTGAC-0000
26 0GACCTGAC--0000
27 0ACCTGAC---0000
28 0CCTGAC----0000
29 0CTGAC----A0000
30 0TGAC----AG0000
31 0GAC----AGT0000
32 0AC----AGTC0000
33 0C----AGTCA0000
34 0----AGTCAG0000
35 0---AGTCAGA0000
36 0--AGTCAGAG0000
37 0-AGTCAGAGG0000
38 0AGTCAGAGGA0000
39 0GTCAGAGGAA0000
40 0TCAGAGGAAA0000
41 0CAGAGGAAAG0000
42 0AGAGGAAAGC0000
43 0GAGGAAAGCA0000
44 0AGGAAAGCAG0000
45 0GGAAAGCAGG0000
46 0GAAAGCAGGG0000
47 0AAAGCAGGGC0000
48 0AAGCAGGGCA0000
49 0AGCAGGGCAT0000
50 0GCAGGGCATA0000
51 0CAGGGCATAG0000
52 0AGGGCATAGA0000
53 